In [8]:
# 데이터 처리
import numpy as np
import pandas as pd
import glob

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 시계열 처리
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 딥러닝 모델
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# CNN도 쓸 경우 대비
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

# 경고 제거
import warnings
warnings.filterwarnings('ignore')

In [10]:
# 모든 월별 파일 불러오기 N_01_extent_v3.0.csv ~ N_12_extent_v3.0.csv
file_paths = sorted(glob.glob("data/N_*_extent_v3.0.csv")) 

# 모든 데이터를 하나로 합치기
df_list = []

for path in file_paths:
    temp = pd.read_csv(path)
    df_list.append(temp)

df_all = pd.concat(df_list, ignore_index=True)

In [11]:
print(df_all.columns)

Index(['year', ' mo', '    data-type', ' region', ' extent', '   area'], dtype='object')


In [12]:
df_all.columns = df_all.columns.str.strip()

In [14]:
# 'mo'를 'month'로
df_all = df_all.rename(columns={"mo": "month"})

# 날짜 컬럼 만들기
df_all['date'] = pd.to_datetime(df_all[['year', 'month']].assign(day=1))

In [18]:
df_all.to_csv("data/arctic_ym.csv", index=False)

In [16]:
# 필요한 컬럼만 선택
df_all = df_all[['date', 'extent', 'area']]

# 날짜 기준 정렬
df_all = df_all.sort_values('date').reset_index(drop=True)

# 결측치 확인
print("결측치 수:\n", df_all.isnull().sum())

# 상위 5개 출력
df_all.head()

결측치 수:
 date      0
extent    0
area      0
dtype: int64


,date,extent,area
0,1978-11-01,11.65,9.04
1,1978-12-01,13.67,10.90
2,1979-01-01,15.41,12.41
3,1979-02-01,16.18,13.18
4,1979-03-01,16.34,13.21


    • extent: 전체 해빙이 퍼져있는 범위 
    • area: 실제 얼음이 차 있는 면적

In [19]:
ao_df = pd.read_csv("data/ao.long.csv")

In [20]:
# 컬럼 이름 재설정 (불필요한 설명 컬럼 제거)
ao_df = ao_df.iloc[:, :2]
ao_df.columns = ['date', 'ao']

# 날짜 타입으로 변환
ao_df['date'] = pd.to_datetime(ao_df['date'])

# 연도 범위 필터링: 1979 ~ 2024
ao_df = ao_df[(ao_df['date'].dt.year >= 1979) & (ao_df['date'].dt.year <= 2024)]

# 인덱스 리셋
ao_df = ao_df.reset_index(drop=True)

# 확인
ao_df.head()

,date,ao
0,1979-01-01,-2.233
1,1979-02-01,-0.697
2,1979-03-01,-0.814
3,1979-04-01,-1.157
4,1979-05-01,-0.250
